In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from IPython.core.debugger import Tracer # debugging
from IPython.display import clear_output
import ipywidgets as widgets
import time

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns # prettify matplotlib

import numpy as np
import sklearn.gaussian_process as gp

In [ ]:
# local modules
import sys; sys.path.append('../src/')
import optimisation as op
import optimisation_gui as op_gui
import synthetic_data
import plot3D

In [ ]:
# make deterministic
np.random.seed(100)

Function to optimize:

In [ ]:
x = np.linspace(-6, 6, 100)
y = np.linspace(-5, 5, 100)
noise = synthetic_data.Noise2D(x, y, 0.3)
f = lambda x,y: 1.5 * (np.sin(0.5*x)**2 * np.cos(y) + 0.1*x + 0.2*y) + noise.get(x,y)
X, Y = np.meshgrid(x, y)
Z = f(X,Y)
best_z = np.min(Z)

In [ ]:
plot3D.surface3D(X,Y,Z)

In [ ]:
ranges = {
    'x': x,
    'y': y,
}
gp_params = dict(
    alpha = 1e-10, # larger => more noise. Default = 1e-10
    # the default kernel
    kernel = 1.0 * gp.kernels.Matern(nu=2.5) + gp.kernels.WhiteKernel(),
    n_restarts_optimizer = 10,
    # make the mean 0 (theoretically a bad thing, see docs, but can help)
    #normalize_y = True,
    copy_X_train = True # whether to make a copy of the training data (in-case it is modified)
)

# important lesson: having a very small close_tolerance but promoting exploration with large kappa gives
# better results than a smaller kappa and larger close_tolerance (for this function at least)
ac_params = {
    'kappa' : 7
}
close_tolerance = 1e-8

if False:
    print('SABOTAGED PARAMETERS: WON\'T DO WELL\n' * 5)
    # sabotage the parameters to show a possible bad configuration that doesn't
    # work well with this problem (but may with others)
    gp_params['kernel'] = 1.0 * gp.kernels.RBF() + gp.kernels.WhiteKernel()
    ac_params = {
        'kappa' : 5
    }
    close_tolerance = 1e-3

In [ ]:
class MyEvaluator(op.Evaluator):
    def test_config(self, config):
        return f(config.x, config.y)

In [ ]:
optimiser = op.BayesianOptimisationOptimiser(ranges,
                                             maximise_cost=False,
                                             acquisition_function='UCB',
                                             acquisition_function_params=ac_params,
                                             gp_params=gp_params,
                                             pre_samples=4,
                                             ac_max_params=None,
                                             close_tolerance=close_tolerance,
                                             allow_parallel=True)
optimiser.step_log_keep = 100
evaluator = MyEvaluator()

In [ ]:
optimiser.run_sequential(evaluator, max_jobs=80)

# or run interactively to see the log output
#task = lambda: optimiser.run_sequential(evaluator, max_jobs=80)
#op_gui.interactive(optimiser, task)

In [ ]:
optimiser.plot_cost_over_time(true_best=best_z);

In [ ]:
def plot2D(n, step):
    optimiser.plot_step_2D('x', 'y', n, true_cost=Z)
op_gui.step_log_slider(optimiser, plot2D, pre_compute=False);

In [ ]:
def plot(n, step):
    optimiser.plot_step_slice('x', n, gp_through_all=True)
op_gui.step_log_slider(optimiser, plot);

In [ ]:
def plot(n, step):
    optimiser.plot_step_slice('y', n, gp_through_all=True)
op_gui.step_log_slider(optimiser, plot);

In [ ]:
optimiser.best_sample()

# Try optimising the same function with random search

In [ ]:
ra = op.RandomSearchOptimiser(ranges, maximise_cost=False)
ra.run_sequential(evaluator, max_jobs=1000)

In [ ]:
ra.plot_cost_over_time(true_best=best_z);

In [ ]:
ra.best_sample()

In [ ]:
ra.scatter_plot('x', 'y')